In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import batoid
import danish
import numpy as np
import yaml

from pathlib import Path

import ipywidgets
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
print(danish.__file__)

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_r.yaml")
intra = telescope.withGloballyShiftedOptic("Detector", [0, 0, -1.5e-3])
extra = telescope.withGloballyShiftedOptic("Detector", [0, 0, +1.5e-3])
wavelength = 620e-9

In [ ]:
_th = 0.0
_ph = 0.0
_angle = 0.0

In [ ]:
@ipywidgets.interact(
    th = ipywidgets.FloatText(value=_th, step=15),
    ph = ipywidgets.FloatText(value=_ph, step=0.05),
    angle = ipywidgets.FloatText(value=_angle, step=15),
)
def f(th, ph, angle):
    global _th, _ph, _angle
    _th = th
    _ph = ph
    _angle = angle
    thx = np.deg2rad(ph) * np.cos(np.deg2rad(th))
    thy = np.deg2rad(ph) * np.sin(np.deg2rad(th))

    intra_zk = batoid.zernikeTA(
        intra,
        thx, thy,
        wavelength,
        jmax = 66,
        eps = 0.612,
        nrad = 10,
        naz = 62,
    ) * wavelength
    fn = Path(danish.__file__).parent / "data" / "RubinObsc.yaml"
    with open(fn) as f:
        mask_params = yaml.safe_load(f)
    # del mask_params["Spider_3D"]

    factory = danish.DonutFactory(
        mask_params=mask_params, pixel_scale=20e-6, spider_angle=angle
    )

    plt.close('all')
    zk = np.zeros_like(intra_zk)
    img = factory.image(
        aberrations = intra_zk + zk,
        thx=thx, thy=thy,
        npix=91
    )
    plt.figure()
    plt.imshow(
        img,
        origin="lower",
    )
    plt.show()